In [3]:
import requests
from bs4 import BeautifulSoup 
import re
import unicodedata
import pythainlp.util
from pythainlp.tokenize import word_tokenize
from pythainlp.util import find_keyword
from pythainlp.util import rank
#from pythainlp.summarize import extract_keywords
from pythainlp.summarize import summarize
import itertools
import sqlite3
from urllib.parse import urljoin
import time
from pythainlp.tag import tag_provinces


In [4]:
class spyder:
    def __init__( self ,target_links,base_url ):
        self.base_url = base_url
        self.target_links = target_links
    
    def get_crawler(self):
        self.result_crawler = self.crawl(self.base_url,0,set())
        return self.result_crawler
    
    def get_check_domain(self):
        self.check_domain_result = self.check_domain(self.base_url,self.get_crawler())
        return self.check_domain_result
    
    def get_check_not_domain(self):
        self.check_not_domain_result = self.check_not_domain(self.base_url,self.get_crawler())   
        return self.check_not_domain_result
    
    def get_check_ref(self):
        self.check_ref_result = self.check_ref(self.get_check_not_domain(),self.target_links)
        return self.check_ref_result
    
    def get_all(self):
        crawl = self.crawl(self.base_url,0,set())
        check_domain =  self.check_domain(self.base_url,crawl) 
        check_not_domain = self.check_not_domain(self.base_url,crawl)
        check_ref = self.check_ref(check_not_domain,self.target_links)
        return check_domain,check_ref
    
    def crawl(self,url, depth,visited):
        if depth < 3 :
            visited.add(url)
            headers = {'User-Agent': "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/42.0.2311.135 Safari/537.36 Edge/12.246"}
            time.sleep(0.3)
            response = requests.get(url,headers=headers)
            try:
                soup = BeautifulSoup(response.text, 'html.parser')
            except:
                soup = BeautifulSoup(response.text, 'lxml')
            links = soup.find_all('a')
            links = [link.get('href') for link in links if link.get('href') and not link.get('href').startswith('#')]
            links = [urljoin(url, link) for link in links if link]

            for link in links:
                if link not in visited:
                    link = link.replace(' ','')
                    visited.add(link)
                    if link.startswith(url):
                        self.crawl(link,depth=depth+1, visited=visited)
        return visited
    
    def check_domain(self,base_url,links):
        result= set()
        for link in links :
            if link.startswith(base_url):
                result.add(link)
        return result
    
    def check_not_domain(self,base_url,links):
        result= set()
        for link in links :
            if not link.startswith(base_url):
                result.add(link)
        return result
    
    def check_ref(self,links,target_links):
        for i in links:
            for j in target_links:
                if i.startswith(j):
                    target_links[j]+=1
        return target_links

In [5]:
def scrape_tags(url):
  response = requests.get(url)
  soup = BeautifulSoup(response.text, 'html.parser')
  try:
    title_tag = soup.find('title').text
  except:
    title_tag = soup.find('title')
  body_tag = soup.find('body')
  text_below_body = body_tag.get_text() 
  body_list =[]
  body_list .append(text_below_body)
  return (body_list,title_tag)


In [6]:
def cleansing(body):
    for i in body:
        output = i.replace('\n', ' ').replace('\xa0', '').replace('®', '').replace(';', '')
        output = " ".join(output.split())
    return output

In [7]:
def get_keyword(body):
    words = body.split()
    word_freq = {}
    for word in words:
        if word in word_freq:
            word_freq[word] += 1
        else:
            word_freq[word] = 1
    return word_freq

In [8]:
body,title = scrape_tags('https://en.wikipedia.org/wiki/Main_Page')
body=cleansing(body)

In [9]:
target_links = {'https://www.bbc.com/news':0}
for i in target_links:
    get_link = spyder(target_links,i)
    target_links = get_link.get_check_ref()
    print(target_links)

{'https://www.bbc.com/news': 0}


In [10]:
def get_doc(target_links):
    doc=[]
    for i in target_links:
        web_spyder=spyder(target_links,i)
        domain_links =web_spyder.get_check_domain()
        for j in domain_links:
            print(j)
            j.replace(" ", "")
            d=dict()    
            body, title = scrape_tags(j)
            body=cleansing(body)
            keyword = get_keyword(body)
            d['link']= j
            d['title'] = title
            d['body']=body
            #d['location']=location
            #d['keyword'] = keyword
            for k in target_links:
                if j.startswith(k):
                    d['ref'] = target_links[k]
            doc.append(d)
            print(d)
    return doc

In [11]:
get_doc(target_links)

https://www.bbc.com/news/world/europe/jersey
{'link': 'https://www.bbc.com/news/world/europe/jersey', 'title': 'Jersey - BBC News', 'body': 'HomepageAccessibility linksSkip to contentAccessibility HelpBBC AccountNotificationsHomeNewsSportWeatheriPlayerSoundsBitesizeCBeebiesCBBCFoodHomeNewsSportReelWorklifeTravelFutureCultureTVWeatherSoundsMore menu Search BBC Search BBC HomeNewsSportWeatheriPlayerSoundsBitesizeCBeebiesCBBCFoodHomeNewsSportReelWorklifeTravelFutureCultureTVWeatherSoundsClose menu BBC NewsNews NavigationSectionsHomeWar in UkraineCoronavirusClimateVideoWorldWorld selectedAsiaUKUK selectedBusinessTechScienceStoriesEntertainment & ArtsHealthWorld News TVIn PicturesReality CheckNewsbeatLong ReadsMore More sectionsEurope selectedJerseyJersey selectedMore More sectionsHomeWar in UkraineCoronavirusClimateVideoWorldWorld selectedWorld HomeAfricaAustraliaEuropeEurope selectedLatin AmericaMiddle EastUS & CanadaAsiaAsia HomeChinaIndiaUKUK selectedUK HomeEnglandN. IrelandScotlandWale